# Multi-Layer Perceptron, MNIST
---
In this notebook, we will train an MLP to classify images from the [MNIST database](http://yann.lecun.com/exdb/mnist/) hand-written digit database.

The process will be broken down into the following steps:
>1. Load and visualize the data
2. Define a neural network
3. Train the model
4. Evaluate the performance of our trained model on a test dataset!

Before we begin, we have to import the necessary libraries for working with data and PyTorch.

In [1]:
# import libraries
import torch
import numpy as np

---
## Load and Visualize the [Data](http://pytorch.org/docs/stable/torchvision/datasets.html)

Downloading may take a few moments, and you should see your progress as the data is loading. You may also choose to change the `batch_size` if you want to load more data at a time.

This cell will create DataLoaders for each of our datasets.

In [11]:
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 10

# choose the training and test datasets

class HamburgerDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self):
        hamdata = np.loadtxt('./vectors.csv',
            delimiter=',', dtype=np.float32)
        self.len = hamdata.shape[0]
        self.info_data = torch.from_numpy(hamdata[:, 0:15])
        self.target_data = torch.from_numpy(hamdata[:, 15:])

    def __getitem__(self, index):
        return self.info_data[index], self.target_data[index]

    def __len__(self):
        return self.len
    
    
class TestDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self):
        hamdata = np.loadtxt('./tester.csv',
            delimiter=',', dtype=np.float32)
        self.len = hamdata.shape[0]
        self.info_data = torch.from_numpy(hamdata[:, 0:15])
        self.target_data = torch.from_numpy(hamdata[:, 15:])

    def __getitem__(self, index):
        return self.info_data[index], self.target_data[index]

    def __len__(self):
        return self.len



# prepare data loaders
dataset = HamburgerDataset()
testset = TestDataset()

train_loader = DataLoader(dataset=dataset,
                                batch_size=batch_size,
                                shuffle=True,
                                num_workers=num_workers)

test_loader =  DataLoader(dataset=dataset,
                                batch_size=batch_size,
                                shuffle=True,
                                num_workers=num_workers)

### Visualize a Batch of Training Data

The first step in a classification task is to take a look at the data, make sure it is loaded in correctly, then make any initial observations about patterns in that data.

### View an Image in More Detail

---
## Define the Network [Architecture](http://pytorch.org/docs/stable/nn.html)

The architecture will be responsible for seeing as input a 784-dim Tensor of pixel values for each image, and producing a Tensor of length 10 (our number of classes) that indicates the class scores for an input image. This particular example uses two hidden layers and dropout to avoid overfitting.

In [12]:
import torch.nn as nn
import torch.nn.functional as F


# define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # number of hidden nodes in each layer (32)    2개의 히든 레이어, 각각 32개의 노드
        hidden_1 = 32
        hidden_2 = 32
        # linear layer (15 -> hidden_1)     15개의 데이터 (연령6, 성별1, 고기종류3, 브랜드4, 매운거선호1)
        self.fc1 = nn.Linear(15, hidden_1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        # linear layer (n_hidden -> 10)       일단 10개의 아웃풋
        self.fc3 = nn.Linear(hidden_2, 10)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data    일단 0.2의 드롭아웃, 0.8만큼 학습
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # flatten image input
        # x = x.view(-1, 28 * 28)   어차피 1차원 이므로 flaten 할 필요 없음
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        # add dropout layer
        x = self.dropout(x)
        # add output layer
       # x = F.relu(self.fc3(x))
        # softmax를 적용하여 반환한다.  => 일단 취소
        return self.fc3(x)

# initialize the NN
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=15, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
  (dropout): Dropout(p=0.2)
)


###  Specify [Loss Function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [Optimizer](http://pytorch.org/docs/stable/optim.html)

It's recommended that you use cross-entropy loss for classification. If you look at the documentation (linked above), you can see that PyTorch's cross entropy function applies a softmax funtion to the output layer *and* then calculates the log loss.

In [13]:
# specify loss function (categorical cross-entropy)
# criterion = nn.CrossEntropyLoss() # crossEntropyLoss는 2차원 인풋만 된다고 해서 다른 loss function 을 사용할 것임
criterion = nn.L1Loss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

---
## Train the Network

The steps for training/learning from a batch of data are described in the comments below:
1. Clear the gradients of all optimized variables
2. Forward pass: compute predicted outputs by passing inputs to the model
3. Calculate the loss
4. Backward pass: compute gradient of the loss with respect to model parameters
5. Perform a single optimization step (parameter update)
6. Update average training loss

The following loop trains for 50 epochs; take a look at how the values for the training loss decrease over time. We want it to decrease while also avoiding overfitting the training data.

In [14]:
# number of epochs to train the model
n_epochs = 50

model = model.to(device)

def train(n_epochs):
    model.train() # prep model for training

    for epoch in range(n_epochs):
        # monitor training loss
        train_loss = 0.0

        ###################
        # train the model #
        ###################
        for data, target in train_loader:
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update running training loss
            train_loss += loss.item()*data.size(0)

        # print training statistics 
        # calculate average loss over an epoch
        train_loss = train_loss/len(train_loader.dataset)

        print('Epoch: {} \tTraining Loss: {:.6f}'.format(
            epoch+1, 
            train_loss
            )) 
        
train(n_epochs)

Epoch: 1 	Training Loss: 0.090656
Epoch: 2 	Training Loss: 0.050880
Epoch: 3 	Training Loss: 0.034559
Epoch: 4 	Training Loss: 0.029662
Epoch: 5 	Training Loss: 0.026418
Epoch: 6 	Training Loss: 0.025848
Epoch: 7 	Training Loss: 0.022860
Epoch: 8 	Training Loss: 0.021503
Epoch: 9 	Training Loss: 0.020164
Epoch: 10 	Training Loss: 0.018738
Epoch: 11 	Training Loss: 0.017404
Epoch: 12 	Training Loss: 0.016298
Epoch: 13 	Training Loss: 0.014779
Epoch: 14 	Training Loss: 0.013900
Epoch: 15 	Training Loss: 0.012971
Epoch: 16 	Training Loss: 0.011801
Epoch: 17 	Training Loss: 0.011235
Epoch: 18 	Training Loss: 0.010033
Epoch: 19 	Training Loss: 0.009112
Epoch: 20 	Training Loss: 0.008569
Epoch: 21 	Training Loss: 0.007424
Epoch: 22 	Training Loss: 0.006750
Epoch: 23 	Training Loss: 0.006553
Epoch: 24 	Training Loss: 0.005953
Epoch: 25 	Training Loss: 0.005155
Epoch: 26 	Training Loss: 0.004962
Epoch: 27 	Training Loss: 0.004642
Epoch: 28 	Training Loss: 0.004235
Epoch: 29 	Training Loss: 0.0

---
## Test the Trained Network

Finally, we test our best model on previously unseen **test data** and evaluate it's performance. Testing on unseen data is a good way to check that our model generalizes well. It may also be useful to be granular in this analysis and take a look at how this model performs on each class as well as looking at its overall loss and accuracy.

In [28]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
correct = 0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval() # prep model for evaluation

for data, target in test_loader:
    output = model(data)
    test_loss += criterion(output,target).data
    pred = output

    correct += abs(pred.sub(target.data.view_as(pred))).sum()
    test_loss /= len(test_loader.dataset)
print('Test set: Average loss: {:.4f}, Accuracy: {}{} ({:.0f}%)'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0000, Accuracy: 2.0435807704925537500 (0%)


### Visualize Sample Test Results

This cell displays test images and their labels in this format: `predicted (ground-truth)`. The text will be green for accurately classified examples and red for incorrect predictions.

In [7]:
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = dataiter.next()

# get sample outputs
output = model(images)
# convert output probabilities to predicted class
_, preds = torch.max(output, 1)
# prep images for display
images = images.numpy()

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    ax.set_title("{} ({})".format(str(preds[idx].item()), str(labels[idx].item())),
                 color=("green" if preds[idx]==labels[idx] else "red"))

NameError: name 'plt' is not defined